This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = "You Don't get to see my key"

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests 


Note: API's can change a bit with each version, for this exercise it is reccomended to use the "V3" quandl api at `https://www.quandl.com/api/v3/`

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2018-10-10&end_date=2018-10-10&api_key=" + API_KEY
request = requests.get(url)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
request.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2018-10-10',
  'end_date': '2018-10-10',
  'data': [['2018-10-10',
    None,
    67.3,
    63.6,
    63.7,
    None,
    215727.0,
    13893065.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

Set the url getting the data for Carl Zeiss Meditec for the year of 2017 and make the request.

In [5]:
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY
request = requests.get(url)

When using the json method on our request object, we get a dictionary containing all sorts of info on our dataset. We will isolate the list of data inside of this dictionary, loop through it and create a new dictionary object for it. The data will be the key for each new day of data, with each piece of data under these keys being assigned the names from the column names from inside the json dictionary. 

In [6]:
data_list = request.json()['dataset']['data']
request.json()['dataset']['column_names']
data_dict = {}
for datapoint in data_list:
    date, OPEN, high, low, close, change, traded_volume, turnover, last_price, traded_units, daily_turnover = datapoint
    data_dict[date] = {'open':OPEN, 'high':high, 'low':low, 'close':close, 'change':change,
                      'traded_volume':traded_volume, 'turnover':turnover, 'last_price_of_the_day':last_price,
                      'daily_traded_units':traded_units, 'daily_turnover':daily_turnover}

After making our dictionary, we set two variables for which we can compare valudes to find the highest and lowest and loop through the dataset until our highest and lowest values are found. 

In [11]:
highest = 0
lowest = 1000000
for date, values in data_dict.items():
    if values['open'] == None : continue
    if values['open'] > highest: highest = values['open']
    if values['open'] < lowest: lowest = values['open']
print("The highest opening price was " + str(highest) +", and the lowest price was " + str(lowest) + ".")

The highest opening price was 53.11, and the lowest price was 34.0.


Loop through the data, finding the change for each day and replacing the highest change if the new chagne was higher. 

In [14]:
highest_change = 0
for date, values in data_dict.items():
    change = values['high'] - values['low']
    if change > highest_change: highest_change = change
print("The highest price change in any one day was " + str(round(highest_change,2)) + ".")

The highest price change in any one day was 2.81.


The days are ordered backwards in this dict, so we will write to code with this in mind. Loop through the data, find the difference between tomorrow's closing price and today's. If this change is higher than highest, replace highest with this new change. 

In [21]:
highest_change = 0
next_day_close = 0
for date, values in data_dict.items():
    if next_day == "": next_day_close = date
    else:
        change = next_day_close - values['close']
        next_day_close = values['close']
        if change > highest_change: highest_change = change
print("The highest change between any two days was " + str(round(highest_change,2)) + ".")
    

The highest change between any two days was 1.72.


Add all the daily volumes from the data set and divide by the length of the data set to find the daily average. There are no null values, so the dict length can be used without issue. 

In [23]:
total = 0
for date, values in data_dict.items():
    total += values['traded_volume']
average = round(total/len(data_dict), 2)
print("The average daily trade volume for the year was " + str(average) + ".")

The average daily trade volume for the year was 89124.34.
